### Libraries

In [1]:
!pip install --upgrade transformers

In [21]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, GPTNeoXForCausalLM
import torch
import numpy as np
from datasets import load_dataset
import pandas as pd

### Policy (model)

In [3]:
model = GPTNeoXForCausalLM.from_pretrained(
  "EleutherAI/pythia-1b-deduped",
  revision="step143000", # main branch
  cache_dir="./pythia-1b-deduped/step143000",
)

tokenizer = AutoTokenizer.from_pretrained(
  "EleutherAI/pythia-1b-deduped",
  revision="step143000",
  cache_dir="./pythia-1b-deduped/step143000",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Reward (model)

In [4]:
reward_model = AutoModelForCausalLM.from_pretrained("cleanrl/EleutherAI_pythia-1b-deduped__reward__tldr")

Some weights of GPTNeoXForCausalLM were not initialized from the model checkpoint at cleanrl/EleutherAI_pythia-1b-deduped__reward__tldr and are newly initialized: ['embed_out.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
reward_model.eval()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2048)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-15): 16 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((2048,), eps=1e-05, 

### Dataset loading



In [6]:
source = "trl-internal-testing/tldr-preference-sft-trl-style"
training_data = load_dataset(source, split="train", download_mode="force_redownload")
test_data = load_dataset(source, split="test", download_mode="force_redownload")
eval_data = load_dataset(source, split="validation", download_mode="force_redownload")

README.md:   0%|          | 0.00/780 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/159M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/160M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/116722 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6447 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6553 [00:00<?, ? examples/s]

In [7]:
test_data.column_names

['prompt', 'messages', 'id', 'subreddit', 'title', 'post', 'summary']

In [8]:
prompt = test_data[0]["prompt"]
inputs = tokenizer(prompt, return_tensors="pt")
generated_tokens = model.generate(**inputs)
generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [17]:
def get_reward(prompt, summary):
    # Format input as expected by the reward model (Reddit-style)
    input_text = f"{prompt.strip()}\n\nTL;DR: {summary.strip()}"

    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        # reward = outputs.logits.squeeze().item()  # scalar reward
        reward = outputs.logits[0, -1, :].mean().item()
        value = outputs.logits[0, -1].mean().item()

    return reward, value


In [18]:
get_reward(test_data[0]["prompt"], generated_text)

(0.00044514506589621305, 0.00044514506589621305)

### Policy evaluation on test examples

In [20]:
def get_generated_summary(prompt):
  inputs = tokenizer(prompt, return_tensors="pt")
  generated_tokens = model.generate(**inputs)
  generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
  return generated_text

In [22]:
reward_df = pd.DataFrame(columns=["prompt", "reward", "value"])

In [23]:
for i in range(100):
  prompt = test_data[i]["prompt"]
  generated_text = get_generated_summary(prompt)
  reward, value = get_reward(prompt, generated_text)
  reward_df.loc[i] = [prompt, reward, value]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for o

In [25]:
reward_df["reward"].max()

1.7664225101470947

In [26]:
reward_df["reward"].min()

-3.3091468811035156